# 5.6 GPU
- **目录**
  - 5.6.1 计算设备
  - 5.6.2 张量与GPU
    - 5.6.2.1存储在GPU上
    - 5.6.2.2 复制
    - 5.6.2.3 旁注
  - 5.6.3 神经网络与GPU

- 自2000年以来，GPU性能每十年增长1000倍。
- 先看看如何使用单个NVIDIA GPU进行计算。
  - 首先，确保你至少安装了一个NVIDIA GPU。
  - 然后，下载[NVIDIA驱动和CUDA](https://developer.nvidia.com/cuda-downloads)，并按照提示设置适当的路径。
- 准备工作完成后可以使用`nvidia-smi`命令来**查看显卡信息。**

In [2]:
!nvidia-smi

Fri Aug 30 20:08:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.13                 Driver Version: 556.13         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX250         WDDM  |   00000000:02:00.0 Off |                  N/A |
| N/A   57C    P0             N/A / ERR!  |       0MiB /   2048MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---------
- **说明：GPU状态信息**
  - NVIDIA-SMI 537.13：所使用的 NVIDIA System Management Interface (SMI) 工具的版本号，它是537.13。
  - Driver Version: 537.13：NVIDIA 显卡驱动程序的版本号为537.13。
  - CUDA Version: 12.2：CUDA（Compute Unified Device Architecture）版本号是12.2，这是 NVIDIA 的并行计算平台和编程模型。
  - GPU 0详细信息：系统的第一块 NVIDIA GeForce GPU 的编号。
  - Name TCC/WDDM：显示了 GPU 的名称和它运行在哪种驱动模式。Windows中，WDDM（Windows Display Driver Model）是显示驱动模式。
  - Bus-Id：GPU 在系统总线上的位置，通常是PCIe总线。
  - Disp.A：显示是否有显示器连接到此 GPU。"Off" 表示没有连接显示器。
  - Volatile Uncorr. ECC：显示 GPU 是否支持 ECC（错误更正码）。这里的"N/A"表示不可用或不支持。
  - Fan：显示 GPU 风扇的当前转速，在这里显示为 "N/A" 可能是因为你的 GPU 没有可监控的风扇，或者是集成到笔记本中。
  - Temp：显示 GPU 的当前温度，这里是44摄氏度。
  - Perf：表示性能状态，P0 表示最高性能状态。
  - Pwr:Usage/Cap：显示 GPU 的当前功耗和功率上限。这里显示为 "N/A"，可能是因为无法监测或者不适用。
  - Memory-Usage：显示 GPU 内存的使用情况，这里显示为0MiB / 2048MiB，表示目前没有使用 GPU 内存。
  - GPU-Util：显示 GPU 的利用率，这里为2%，意味着 GPU 几乎处于空闲状态。
  - Compute M.：显示 GPU 的计算模式，默认模式是"Default"。
  - MIG M.：MIG（Multi-Instance GPU）模式，用于数据中心的 GPU 分区。这里显示为"N/A"，表明不适用或不启用。
  - Processes：显示当前在 GPU 上运行的进程。由于没有进程在使用 GPU，所以这里没有列出任何内容。
----------------

- 在PyTorch中，每个数组(张量)都有一个设备（device）属性，我们通常将其称为上下文（context）。
- 默认情况下，所有变量和相关的计算都分配给CPU。
- 有时上下文可能是GPU。
- 当跨多个服务器部署作业时，事情会变得更加棘手。
- 通过智能地将数组分配给上下文，
我们可以最大限度地减少在设备之间传输数据的时间。
    - 例如，当在带有GPU的服务器上训练神经网络时，
我们通常希望**模型的参数在GPU上**。


## 5.6.1 计算设备
- 可以指定用于存储和计算的设备，如CPU和GPU。
- 默认情况下，张量是在内存中创建的，然后使用CPU计算它。


- 在PyTorch中，CPU和GPU可以用`torch.device('cpu')`
和`torch.device('cuda')`表示。
- 注意：`cpu`设备意味着所有物理CPU和内存，
这意味着PyTorch的计算将尝试使用所有CPU核心。
- 然而，`gpu`设备只代表一个卡和相应的显存。
如果有多个GPU，使用`torch.device(f'cuda:{i}')`
来表示第$i$块GPU（$i$从0开始）。
- 另外，<b>`cuda:0`和`cuda`是等价</b>的。


In [3]:
%matplotlib inline
import torch
torch.cuda.current_device()

0

In [4]:
from torch import nn
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:0')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=0))

- **查询可用gpu的数量。**


In [5]:
torch.cuda.device_count()

1

- 定义了两个方便的函数，**这两个函数允许在不存在GPU的情况下运行代码。**


In [6]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(0), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cuda', index=0),
 [device(type='cuda', index=0)])

## 5.6.2 张量与GPU

- 可以查询**张量所在设备。**
- 默认情况下，张量是在CPU上创建的。


In [7]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

- 需要注意的是，无论何时我们要对多个项进行操作，
它们都必须在同一个设备上。
- 例如，如果我们对两个张量求和，
我们需要确保两个张量都位于同一个设备上，
否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

###  5.6.2.1存储在GPU上

- 可在创建张量时指定存储设备。
- 接下来在第一个`gpu`上创建张量变量`X`。
- 在GPU上创建的张量只消耗这个GPU的显存。
- 需要确保不创建超过**GPU显存限制**的数据。

In [8]:
## 注意device参数的用法
X = torch.ones(2, 3, device=try_gpu(0))
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

- 假设你至少有两个GPU，下面的代码将在**第二个GPU上创建一个随机张量。**


In [9]:
## 本课程的代码运行平台只有一个GPU
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.0430, 0.4509, 0.1126],
        [0.8580, 0.3325, 0.9666]], device='cuda:0')

In [10]:
## device的值可以直接使用字符串代替，也就是设备名，此处为'cuda:0'
Y1 = torch.rand(2, 3, device='cuda:0')
Y1

tensor([[0.1245, 0.2988, 0.7855],
        [0.9729, 0.3611, 0.0119]], device='cuda:0')

### 5.6.2.2 复制

- 如果**要计算`X + Y`，我们需要决定在哪里执行这个操作**。
  - 例如，如图5.6.1所示，我们可以将`X`传输到第二个GPU并在那里执行操作。
- 切记：不要简单地`X`加上`Y`，因为这会导致异常，运行时引擎不知道该怎么做：它**在同一设备上找不到数据会导致失败**。
- 由于`Y`位于第二个GPU上，所以我们需要将`X`移到那里，然后才能执行相加运算。

<center><img src=../img/copyto.svg></center>
<center>图5.6.1 复制数据以便在同一设备上执行操作</center>


In [11]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


- 现在数据在同一个GPU上（`Z`和`Y`都在），可以将它们相加。


In [12]:
Y + Z

tensor([[1.0430, 1.4509, 1.1126],
        [1.8580, 1.3325, 1.9666]], device='cuda:0')

- 假设变量`Z`已经存在于第二个GPU上，如果仍调用`Z.cuda(1)`会发生什么？
- 它将返回`Z`，而不会复制并分配新内存。


In [13]:
Z.cuda(0) is Z

True

### 5.6.2.3 旁注

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。
但是**在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多**。
这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收），
然后才能继续进行更多的操作。
这就是为什么**拷贝操作要格外小心**。
根据经验，**多个小操作比一个大操作糟糕得多**。
此外，一次执行几个操作比代码中散布的许多单个操作要好得多（除非你确信自己在做什么）。
如果一个设备必须等待另一个设备才能执行其他操作，
那么这样的操作可能会阻塞。
这有点像排队订购咖啡，而不像通过电话预先订购：
当你到店的时候，咖啡已经准备好了。

最后，当我们打印张量或将张量转换为NumPy格式时，
如果数据不在内存中，框架会首先将其复制到内存中，
这会导致**额外的传输开销**。
更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

- **要点：**
  - 使用GPU进行机器学习可以加速运算，因为单个GPU相对运行速度快。
  - 在不同设备（如CPU、GPU）之间传输数据比计算慢得多，这可能导致性能瓶颈。
  - 并行化可能受到数据传输延迟的影响，需要等待数据发送和接收才能执行更多操作。
  - 小心拷贝操作，因为多个小操作通常比一个大操作效果更差。
  - 一次执行几个操作通常比代码中散布的许多单个操作效果更好。
  - 设备之间的阻塞操作会影响性能，类似于排队订购咖啡而不是提前预订。
  - 当打印张量或将张量转换为NumPy格式时，如果数据不在内存中，框架会首先将其复制到内存中，导致额外的传输开销。
  - 在执行此类操作时，Python的全局解释器锁可能会导致进一步的性能下降，因为所有其他操作都必须等待Python完成。

## 5.6.3 神经网络与GPU

- 为神经网络模型指定设备。 
- 下面的代码将模型参数放在GPU上。

In [14]:
## 将模型移到GPU上
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net

Sequential(
  (0): Linear(in_features=3, out_features=1, bias=True)
)

- 当输入为GPU上的张量时，模型将在同一GPU上计算结果。


In [15]:
net(X)

tensor([[0.8714],
        [0.8714]], device='cuda:0', grad_fn=<AddmmBackward0>)

- 确认模型参数存储在同一个GPU上。

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)

## 小结

* 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。
* 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
* 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy `ndarray`中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。